#  Stock Portfolio Hedge

## Members

Marcos Cavalcanti - 1920533

Lucas Mossi - 2010618

## Install Libraries

In [1]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
from sklearn.linear_model import LinearRegression

C:\Users\letic\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
if mt5.initialize():
    print("Conectado")
else:
    print("initialize() failed")
    mt5.shutdown()

Conectado


## Import Data - Yahoo Finance

In [3]:
tickers = ["PETR4.SA","VALE3.SA","ITUB4.SA","EGIE3.SA"]
start_date = "2022-01-01"
end_date = "2022-05-31"


for i,ticker in enumerate(tickers):
    if i == 0:
        iteration = pd.DataFrame(yf.download(ticker,start = start_date,end = end_date))["Open"]
        stocks = pd.DataFrame(index = iteration.index)
        stocks[f"{ticker[:5]}_open"] = np.array(iteration)
        iteration = pd.DataFrame(yf.download(ticker,start = start_date,end = end_date))["Close"]
        stocks[f"{ticker[:5]}_close"] =np.array(iteration)
    else:
        iteration = pd.DataFrame(yf.download(ticker,start = start_date,end = end_date))["Open"]
        stocks[f"{ticker[:5]}_open"] = np.array(iteration)
        iteration = pd.DataFrame(yf.download(ticker,start = start_date,end = end_date))["Close"]
        stocks[f"{ticker[:5]}_close"] =np.array(iteration)
stocks

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,PETR4_open,PETR4_close,VALE3_open,VALE3_close,ITUB4_open,ITUB4_close,EGIE3_open,EGIE3_close
Date,,,,,,,,
2022-01-03,28.540001,29.090000,79.000000,78.000000,21.170000,21.510000,38.500000,38.040001
2022-01-04,29.160000,29.200001,78.510002,77.080002,21.620001,22.120001,38.040001,38.139999
2022-01-05,29.190001,28.070000,77.370003,77.809998,22.090000,21.700001,38.130001,37.599998
2022-01-06,28.290001,28.049999,78.639999,79.379997,21.639999,22.139999,37.680000,37.299999
2022-01-07,28.110001,28.180000,80.400002,84.000000,22.090000,22.629999,37.290001,37.349998
...,...,...,...,...,...,...,...,...
2022-05-24,31.200001,31.600000,81.820000,84.040001,25.680000,26.170000,43.770000,44.840000
2022-05-25,31.700001,32.049999,84.000000,84.300003,26.049999,25.700001,44.700001,45.169998
2022-05-26,32.020000,32.130001,84.160004,84.250000,25.730000,26.020000,45.160000,44.110001


## Import Data - Investing.com

In [4]:
future = pd.read_csv("C:\\Users\\letic\\Downloads\\win_2022-01-03_2022-05-31.csv")
future = future[["Data","Último","Abertura"]].astype(str)
future["Date"] = np.nan

for i in range(len(future)):
    future.loc[i,["Date"]] = str(future.loc[i,["Data"]])[14:18]+"-"+str(future.loc[i,["Data"]])[11:13]+"-"+str(future.loc[i,["Data"]])[8:10]

future["Close"] = np.array(future["Último"]).astype(float)*1000
future["Open"] = np.array(future["Abertura"]).astype(float)*1000    
future = future[["Date","Open","Close"]]
future.set_index(["Date"],inplace=True)
future

,Open,Close
Date,,
2022-05-31,111290.0,111553.0
2022-05-30,113215.0,111546.0
2022-05-27,112300.0,112489.0
2022-05-26,111500.0,112536.0
2022-05-25,110815.0,111280.0
...,...,...
2022-01-07,102385.0,103333.0
2022-01-06,101975.0,102349.0
2022-01-05,104570.0,101839.0


## Questions 01 / 02

In [5]:
# Initial Capital: R$400.000,00
initial_capital = 400000


# Portfolio Allocation
cota = initial_capital / (len(stocks.columns)/2)
tickers = ["PETR4","VALE3","ITUB4","EGIE3"]
portfolio = dict.fromkeys(tickers,0)
value_mar = 0


# Amount Definition
for i,ticker in enumerate(tickers):
    sum_ticker = 0
    while (sum_ticker < cota):
        delta = float(100 * stocks.loc["2022-03-31",[f"{ticker}_open"]])
        if delta > (cota - sum_ticker):
            break
        else:
            sum_ticker  += delta
            portfolio[f"{ticker}"] += np.array([float(100),float(0)])
    value_mar += sum_ticker
    portfolio[f"{ticker}"] += np.array([float(0),float(sum_ticker)])
    cota = (initial_capital/len(tickers)) + (cota - sum_ticker)

# Results
print("Ticker:  [ Amount             Value     ]")
for ticker in tickers:
    print(f"{ticker} : ",portfolio[ticker])
print("")
print("Value of the portfolio in March  = %.2f \n "%(value_mar))
print("Residual given to the Broker = %.2f" %(initial_capital - value_mar))

Ticker:  [ Amount             Value     ]
PETR4 :  [ 3000.         97709.99908447]
VALE3 :  [ 1000.         96419.99816895]
ITUB4 :  [  3700.         103155.99689484]
EGIE3 :  [ 2300.         99682.00035095]

Value of the portfolio in March  = 396967.99 
 
Residual given to the Broker = 3032.01


## Question 03

In [6]:
# Value of the portfolio in 2022-04-29
value_apr = 0
for ticker in portfolio.keys():
    value_apr += float(float(portfolio[ticker][0]) * float(stocks.loc["2022-04-29",[f"{ticker}_open"]]))

# Results
print("Value of the portfolio in April = %.2f"%(value_apr))

Value of the portfolio in April = 367556.00


## Question 04

Let $\; X^{T} = [x_1 ... x_m] \;$ and $\;Y^{T} = [y_1 ... y_m]\;$ be the vector of the returns from an asset and an index, respectively, then the estimated beta ($\hat\beta$) between both is given by

$\hat\beta = \frac{\left( x_i - \frac{1}{m}\sum\limits_{i=1}^{m}x_i \right) \left( y_i - \frac{1}{m}\sum\limits_{i=1}^{m}y_i \right) }{\sum\limits_{i=1}^{m} \left( y_i - \frac{1}{m}\sum\limits_{i=1}^{m}y_i \right) ^2 }$

In that way, we will estimate the beta from each stock belonging to the portfolio and the future index of that same market.

Along with the demonstration in "Extra 01" (at the end of the document) the beta from a portfolio is a linear composition of its assets' betas, that is

$\hat\beta = \sum\limits_{j=1}^{n} \omega_j \hat\beta_{j} \;$, where $\omega_j$ and $\hat\beta_{j}$ are the weight and the estimated beta of the $j_{th}$ asset of the portfolio, respectively.

We selected the 90 previous trading days to tackle this task.

In [27]:
trading_days = 90
stocks_indexes = stocks[stocks.index <= "2022-04-28"].index[-trading_days:]
future_indexes = future[future.index <= "2022-04-28"].index[-trading_days:]
omega = dict.fromkeys(tickers,0)
betas = dict.fromkeys(tickers+["portfolio"],0)

for ticker in tickers:
    X = list()
    y = list()
    for i in range(len(stocks_indexes)-1):
        X.append(np.log( float(stocks.loc[stocks_indexes[-i-1],[f"{ticker}_close"]]) / float(stocks.loc[stocks_indexes[-i-2],[f"{ticker}_close"]])))
        y.append(np.log( float(future.loc[future_indexes[-i-1],["Close"]]) / float(future.loc[future_indexes[-i-2],["Close"]])))
    reg = LinearRegression().fit(np.array(X).reshape(-1, 1) , np.array(y))
    betas[ticker] = reg.coef_

for ticker in tickers:
    betas["portfolio"] += float( ( ( float(float(portfolio[ticker][0]) * float(stocks.loc["2022-04-29",[f"{ticker}_open"]])) ) / value_apr ) * float(betas[ticker][0])) 
    print(f"{ticker}: %.5f "  %(betas[ticker]))
print("Portfolio : %.5f" %(betas["portfolio"]))

PETR4: -0.00585 
VALE3: 0.02173 
ITUB4: 0.05548 
EGIE3: 0.12117 
Portfolio : 0.04980


## Questions 05 / 06

What about the number of contracts chosen and whether we buy or sell them? 

Well...

Firstly, we must define the function G which represents the gains from the hedge operation. Consider an asset $S$, a future contract $F$ and that each individual asset represents a future contract.

$G: \mathbb{N}\; \rightarrow \mathbb{R}$

$ \;\;\;\;\; N \; \mapsto  1 \times \left( S(t_2) - S(t_1) \right) - N \left( F(t_2,T) - F(t_1,T) \right) \;=\; 1 \times \Delta S - N \times \Delta F$

Furthermore, we should solve this problem is respect of the following optimization problem:

Given 

$\sigma^2 \left(G(N)\right) \;=\; \sigma^2(\Delta S) + N^2 \times \sigma^2 (\Delta F) - 2 \times N \times cov(\Delta S, \Delta F) =\;$

$\;=\; \sigma^2(\Delta S) + N^2 \times \sigma^2 (\Delta F) - 2 \times N \times \rho \times \sigma(\Delta S) \times \sigma(\Delta F) \;$ .

We aim to

$\min_{N} \sigma^2(G) \; \rightarrow$

$ \rightarrow\; \frac{\partial \sigma^2 (G)}{\partial h} = 0$

$ \rightarrow\;    N^{*} \;=\; \frac{cov\left(\Delta S, \Delta F\right)}{\sigma^2(\Delta F)}$

which is the estimation of the linear dependence of the asset from the future.

Statiscally speaking, compute $N^{*}$ is equivalent to calculate the beta of the followiing expression:

$\Delta S = \alpha + \beta \times \Delta F + \epsilon \;$

Finally, 

$N^* = \hat\beta \times Q $ , where $Q$ is the future contract financial amount equivalent to the underlying asset.  

In [28]:
N = value_apr / ( float(future.loc["2022-04-29",["Open"]]) * 0.20),future.loc["2022-04-29",["Open"]]
print("%.5f contracts @ %.2f" %(N))
N = round(N[0])

16.58048 contracts @ 110840.00


Consequently, we choose to $\textbf{sell}$ 17 contracts, once the portfolio has a $\textbf{positive}$ linear relation with the future index and it must be an intenger number.

## Question 07

Considering the linearity demonstrated in "Extra 01", and provided that all the conditions are the same, the numbers of contracts taken must be reduced by one-third.

## Question 08

#### Stock Portfolio Without Hedge

In [29]:
value_may = 0
for ticker in tickers:
    value_may += float(portfolio[f"{ticker}"][0])*float(stocks.loc["2022-05-30",[f"{ticker}_close"]])

print("Value in April: %.2f \n" %(value_apr))
print("Value in May: %.2f \n" %(value_may))
print("Variation of the portfolio without hedge (2022-04-29 / 2022-05-30): %.2f \n" %(value_may - value_apr))

Value in April: 367556.00 

Value in May: 371930.00 

Variation of the portfolio without hedge (2022-04-29 / 2022-05-30): 4374.00 



#### Stock Portfolio With Hedge

In [30]:
hedge = N * ( float(future.loc["2022-04-29",["Open"]]) - float(future.loc["2022-05-30",["Close"]]) )

print("Gain / Loss exclusely with hedging: %.2f \n"%(hedge))
print("Variation of the portfolio with hedge (2022-04-29 / 2022-05-30): %.2f \n" %(value_may - value_apr + hedge))

Gain / Loss exclusely with hedging: -12002.00 

Variation of the portfolio with hedge (2022-04-29 / 2022-05-30): -7628.00 



#### Daily Settlement

In [31]:
daily_settlement = future[(future.index >= "2022-04-29") & (future.index <= "2022-05-30")]
daily_settlement["daily_settlement"] = np.nan
indexes = daily_settlement.index[::-1]

for i in range(len(indexes)):
    daily_settlement.loc[indexes[i],["daily_settlement"]] = N * (float(daily_settlement.loc[indexes[i],["Open"]]) - float(daily_settlement.loc[indexes[i],["Close"]]))


print("Daily Settlement considering the selling of %d contracts\n" %(N))
print(daily_settlement)

Daily Settlement considering the selling of 17 contracts

                Open     Close  daily_settlement
Date                                            
2022-05-30  113215.0  111546.0           28373.0
2022-05-27  112300.0  112489.0           -3213.0
2022-05-26  111500.0  112536.0          -17612.0
2022-05-25  110815.0  111280.0           -7905.0
2022-05-24  110200.0  111252.0          -17884.0
2022-05-23  110050.0  111365.0          -22355.0
2022-05-20  108650.0  109323.0          -11441.0
2022-05-19  106580.0  107708.0          -19176.0
2022-05-18  109260.0  106987.0           38641.0
2022-05-17  110310.0  109836.0            8058.0
2022-05-16  107575.0  109270.0          -28815.0
2022-05-13  108065.0  108041.0             408.0
2022-05-12  104460.0  106893.0          -41361.0
2022-05-11  105275.0  105438.0           -2771.0
2022-05-10  105185.0  104152.0           17561.0
2022-05-09  104790.0  104489.0            5117.0
2022-05-06  106150.0  106266.0           -1972.0
2022-05-05 

C:\Users\letic\AppData\Local\Temp/ipykernel_50792/659837440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_settlement["daily_settlement"] = np.nan
C:\Users\letic\anaconda\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\letic\anaconda\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

## Question 09

Firstly, the stocks belonging to the portfolio gained value in the hedging period. By consequence, the hedge was a detractor of performance. The question is: by how much? As we can see, the lost with the hedge was superior to the gain in the stock market.

In [32]:
print("Gain in the stock market: %2.f \n"%(value_may - value_apr))

print("Loss in the hedge %.2f \n"%(hedge))

print("Net loss: %.2f"%(value_may - value_apr + hedge))

Gain in the stock market: 4374 

Loss in the hedge -12002.00 

Net loss: -7628.00


It happened what is generally called as $\textbf{over-hedging}$. The reason is due to the relatively large uncertainty with the beta estimator ($\hat\beta$). If we compare the estimated beta with the realized one, it can be seen that the latter has significantly increased.

In [34]:
tickers = ['PETR4', 'VALE3', 'ITUB4', 'EGIE3']
stocks_indexes = stocks[(stocks.index >= "2022-04-28")&(stocks.index <= "2022-05-30")].index
future_indexes = future[(future.index >= "2022-04-28")&(future.index <= "2022-05-30")].index
omega = dict.fromkeys(tickers,0)
betas_realized = dict.fromkeys(tickers+["portfolio"],0)

for ticker in tickers:
    X = list()
    y = list()
    for i in range(len(stocks_indexes)-1):
        X.append(np.log( float(stocks.loc[stocks_indexes[-i-1],[f"{ticker}_close"]]) / float(stocks.loc[stocks_indexes[-i-2],[f"{ticker}_close"]])))
        y.append(np.log( float(future.loc[future_indexes[-i-1],["Close"]]) / float(future.loc[future_indexes[-i-2],["Close"]])))
    reg = LinearRegression().fit(np.array(X).reshape(-1, 1) , np.array(y))
    betas_realized[ticker] = reg.coef_

for ticker in tickers:
    betas_realized["portfolio"] += float( ( ( float(float(portfolio[ticker][0]) * float(stocks.loc["2022-04-29",[f"{ticker}_open"]])) ) / value_apr ) * float(betas_realized[ticker][0])) 
    print(f"{ticker} realized beta: %.5f "  %(betas[ticker]))
print("")
print("Portfolio's realized beta = %.5f > %.5f = Portfolio's estimated beta" %(betas_realized["portfolio"],betas["portfolio"]))

PETR4 realized beta: -0.00585 
VALE3 realized beta: 0.02173 
ITUB4 realized beta: 0.05548 
EGIE3 realized beta: 0.12117 

Portfolio's realized beta = 0.07575 > 0.04980 = Portfolio's estimated beta


Hedging does not necessarily mean making (ex post) a good deal.

In that specific case that the market went in favor of the investor the hedge was not a good option. Howerver, if the opposite had happened, the bias would have allowed the investor to profit from the hedge operation.

## Extra 01

### Prove that $\;\hat\beta = \sum\limits_{j=1}^{n} \omega_j \hat\beta_j\;$ where $\;\hat\beta\;$ is the betta of the portfolio, $\omega_j$ is the weight of the $j_{th}$ asset and $\hat\beta_j$ is the betta of the $j_{th}$ asset.

### Analytical Proof

Consider a market with $n$ assets over $m$ observations. Then, take the returns of each time series as $\bold{x}^{T}_i = \left[\;x_{1i} \;...\;x_{mi}\;\right] \in \mathbb{R}^{{m}} , i \in \{1,...,n\}$ . Subsequently, we have

$X =  \begin{bmatrix}
x_{11}& x_{12} & ... & x_{1n} \\
x_{21} & x_{22} & ... & x_{2n} \\
 :& : & : & : \\
x_{m1} & x_{m2} & ... & x_{mn}
\end{bmatrix}\;$.

Define the function $cov: \mathbb{M}_{mXn}(\mathbb{R})\rightarrow \mathbb{M}_{nXn}(\mathbb{R})$ which estimates the covariance matrice.

$\Sigma = \begin{bmatrix} 
\sigma_{1}^2& \sigma_{12} & ... & \sigma_{1n} \\
\sigma_{21} & \sigma_{2}^2 & ... & \sigma_{2n} \\
 :& : & : & : \\
\sigma_{n1} & \sigma_{n2} & ... & \sigma_{n}^2
\end{bmatrix}\; , \sigma_{ij} = \sum\limits_{k = 1}^{m}\frac{(x_{ki} - \bar{x}_i)(x_{kj} - \bar{x}_j)}{m - 1} \;,\; \bar{x}_i = \frac{1}{m}\sum\limits_{k = 1} x_{ki} \;,\; i,j \in \{1,...n\}$

In addition, define a same-length vector $Y^T = \left[\;y_{1}\; ... \;y_m\;\right]$, which represents the returns of the market, then

$\hat\beta_j = \frac{ \left(\sum\limits_{i=1}^{m} x_{ij} - \frac{1}{m}\sum\limits_{i=1}^{m} x_{ij} \right) \left(\sum\limits_{i=1}^{m} y_{i} - \frac{1}{m}\sum\limits_{i=1}^{m} y_{i} \right) }{\left(\sum\limits_{i = 1}^{m} y_i - \frac{1}{m}\sum\limits_{i=1}^{m}y_i\right)^2}\;, j \in \{1,...,n\}\;\;$ is the betta of the $j$ asset of the market.

Suppose the following portfolio composed by the vector of weights $\omega^{T} =  \left[\;\omega_1\ ...\;\omega_n \;\right] \in [0,1]^n\;$, such that $ \;\sum\limits_{i = 1}^{n}\omega_i = 1$. Then,

$\hat\beta = \frac{ \left(\sum\limits_{i=1}^{m} \sum\limits_{j=1}^{n} \omega_j x_{ij} - \frac{1}{m}\sum\limits_{i=1}^{m} \sum\limits_{j=1}^{n} \omega_j x_{ij} \right) \left(\sum\limits_{i=1}^{m} y_{i} - \frac{1}{m}\sum\limits_{i=1}^{m} y_{i} \right) }{\left(\sum\limits_{i = 1}^{m} y_i - \frac{1}{m}\sum\limits_{i=1}^{m}y_i\right)^2}\;\;$ is the betta of the portfolio.

Prove that $\hat\beta = \sum\limits_{j=1}^{n}\omega_j \hat\beta_j\;$.

Demonstration:

$\sum\limits_{j=1}^{n} \omega_j \hat\beta_j \;=\; \sum\limits_{j=1}^{n} \omega_j \frac{ \left( \sum\limits_{i=1}^{m}x_{ij} - \frac{1}{m} \sum\limits_{i=1}^{m} x_{ij} \right)  \left( \sum\limits_{i=1}^{m}y_i - \frac{1}{m} \sum\limits_{i=1}^{m} y_i \right) } { \left( \sum\limits_{i=1}^{m}y_i - \frac{1}{m} \sum\limits_{i=1}^{m} y_i \right)^2}$

$\sum\limits_{j=1}^{n} \omega_j \hat\beta_j \;=\;  \frac{  \sum\limits_{j=1}^{n} \left( \sum\limits_{i=1}^{m}\omega_j  x_{ij} - \frac{1}{m} \sum\limits_{i=1}^{m} \omega_j x_{ij} \right)  \left( \sum\limits_{i=1}^{m}y_i - \frac{1}{m} \sum\limits_{i=1}^{m} y_i \right) } { \left( \sum\limits_{i=1}^{m}y_i - \frac{1}{m} \sum\limits_{i=1}^{m} y_i \right)^2}$

$\sum\limits_{j=1}^{n} \omega_j \hat\beta_j \;=\;  \frac{ \left( \sum\limits_{j=1}^{n} \sum\limits_{i=1}^{m}\omega_j  x_{ij} - \frac{1}{m}  \sum\limits_{j=1}^{n}  \sum\limits_{i=1}^{m} \omega_j x_{ij} \right)  \left( \sum\limits_{i=1}^{m}y_i - \frac{1}{m} \sum\limits_{i=1}^{m} y_i \right) } { \left( \sum\limits_{i=1}^{m}y_i - \frac{1}{m} \sum\limits_{i=1}^{m} y_i \right)^2}$

$\sum\limits_{j=1}^{n} \omega_j \hat\beta_j \;=\;  \frac{ \left( \sum\limits_{i=1}^{n} \sum\limits_{j=1}^{m}\omega_j  x_{ij} - \frac{1}{m}  \sum\limits_{i=1}^{n}  \sum\limits_{j=1}^{m} \omega_j x_{ij} \right)  \left( \sum\limits_{i=1}^{m}y_i - \frac{1}{m} \sum\limits_{i=1}^{m} y_i \right) } { \left( \sum\limits_{i=1}^{m}y_i - \frac{1}{m} \sum\limits_{i=1}^{m} y_i \right)^2}$

$\sum\limits_{j=1}^{n} \omega_j \hat\beta_j \;=\; \hat\beta $

$\;\;\;\;\;\;\;\;\;\;\;\;\;\blacksquare$

### Empirical Proof

#### Import Data

In [35]:
tickers = ["PETR4.SA","VALE3.SA","ITUB4.SA","EGIE3.SA","^BVSP"]
start_date = "2021-01-01"
end_date = "2021-12-31"

for i,ticker in enumerate(tickers):
    if i == 0:
        data = pd.DataFrame(yf.download(ticker,start_date,end_date))['Close'].rename(f'{ticker}')
    else:
        iteration = pd.DataFrame(yf.download(ticker,start_date,end_date))['Close'].rename(f'{ticker}')
        data = pd.merge(data,iteration,how = 'left',left_index= True,right_index= True)
        
data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,PETR4.SA,VALE3.SA,ITUB4.SA,EGIE3.SA,^BVSP
Date,,,,,
2021-01-04,28.910000,91.459999,30.900000,43.580002,118558.0
2021-01-05,30.040001,93.000000,30.700001,43.189999,119223.0
2021-01-06,30.100000,96.050003,31.600000,42.840000,119851.0
2021-01-07,31.000000,102.320000,32.830002,42.480000,121956.0
2021-01-08,31.120001,102.000000,32.820000,44.259998,125077.0
...,...,...,...,...,...
2021-12-23,28.330000,79.150002,21.340000,38.599998,104891.0
2021-12-27,28.750000,78.949997,21.559999,38.650002,105531.0
2021-12-28,28.780001,77.050003,21.480000,38.799999,104864.0


#### Returns

In [36]:
indexes = data.index
tickers_old = ["PETR4.SA","VALE3.SA","ITUB4.SA","EGIE3.SA","^BVSP"]
tickers = ["PETR4","VALE3","ITUB4","EGIE3","IBOV"]

for j in range(len(tickers)):
    data[tickers[j]] = np.nan

for i in range(len(data)-1):
    for j,ticker in enumerate(tickers):
        data.loc[indexes[i+1],[ticker]] = ( float(data.loc[indexes[i+1],[tickers_old[j]]]) - float(data.loc[indexes[i],[tickers_old[j]]]) ) / float(data.loc[indexes[i],[tickers_old[j]]])

data = data.dropna()
data = data[tickers]
data

,PETR4,VALE3,ITUB4,EGIE3,IBOV
Date,,,,,
2021-01-05,0.039087,0.016838,-0.006472,-0.008949,0.005609
2021-01-06,0.001997,0.032796,0.029316,-0.008104,0.005267
2021-01-07,0.029900,0.065278,0.038924,-0.008403,0.017563
2021-01-08,0.003871,-0.003127,-0.000305,0.041902,0.025591
2021-01-11,-0.008355,-0.000196,-0.022547,-0.019431,-0.018149
...,...,...,...,...,...
2021-12-23,0.006037,-0.009635,0.005181,-0.001810,-0.003354
2021-12-27,0.014825,-0.002527,0.010309,0.001295,0.006102
2021-12-28,0.001044,-0.024066,-0.003711,0.003881,-0.006320


#### Equal-Weighted Portfolio

In [37]:
indexes = data.index
omega = [0.25,0.25,0.25,0.25]
data["portfolio"] = np.zeros(len(data))

for index in indexes:
    for j,ticker in enumerate(tickers[:-1]):
        data.loc[index,["portfolio"]] +=  float(omega[j]) * float(data.loc[index,[ticker]])

data = data[tickers[:-1]+["portfolio"]+tickers[-1:]]
data

,PETR4,VALE3,ITUB4,EGIE3,portfolio,IBOV
Date,,,,,,
2021-01-05,0.039087,0.016838,-0.006472,-0.008949,0.010126,0.005609
2021-01-06,0.001997,0.032796,0.029316,-0.008104,0.014001,0.005267
2021-01-07,0.029900,0.065278,0.038924,-0.008403,0.031425,0.017563
2021-01-08,0.003871,-0.003127,-0.000305,0.041902,0.010585,0.025591
2021-01-11,-0.008355,-0.000196,-0.022547,-0.019431,-0.012632,-0.018149
...,...,...,...,...,...,...
2021-12-23,0.006037,-0.009635,0.005181,-0.001810,-0.000057,-0.003354
2021-12-27,0.014825,-0.002527,0.010309,0.001295,0.005976,0.006102
2021-12-28,0.001044,-0.024066,-0.003711,0.003881,-0.005713,-0.006320


#### Covariances Matrix Function

In [38]:
def covariances_matrix_function(data):
    mat = np.array(data)
    m,n = np.shape(mat)
    sigma = np.zeros((n,n))
    for j2 in range(n):
        for j1 in range(n):
            for i in range(m):
                sigma[j1,j2] += (mat[i,j1] - np.mean(mat[:,j1])) * (mat[i,j2] - np.mean(mat[:,j2])) / (m-1)    
    return sigma        

#### Portfolio's Beta

In [39]:
sigma = covariances_matrix_function(data)
tickers = ["PETR4","VALE3","ITUB4","EGIE3","portfolio"]
betas_extra = dict.fromkeys(tickers,0)

for j,ticker in enumerate(tickers):
    betas_extra[ticker] = float(sigma[j,len(sigma)-1]) / sigma[len(sigma)-1,len(sigma)-1]

#### Sum Betas

In [40]:
sum_betas = 0
for j,ticker in enumerate(tickers[:-1]):
    sum_betas += omega[j] * betas_extra[ticker]

$\hat\beta = \sum\limits_{j=1}^{n}\omega_j \hat\beta_j\;$ ??

In [41]:
print("Proportional sum of betas: %f \n"%(sum_betas))
print("Portfolio's beta: %f"%(betas_extra["portfolio"]))

Proportional sum of betas: 0.960687 

Portfolio's beta: 0.960687


As we wanted to (empirically) proof!

Nevertheless, I could found a marginal difference when I first used $\textbf{logarithmic}$ returns, instead of linear ones. Let's check it!

#### Logarithmic Returns

In [42]:
tickers = ["PETR4.SA","VALE3.SA","ITUB4.SA","EGIE3.SA","^BVSP"]
start_date = "2021-01-01"
end_date = "2021-12-31"

for i,ticker in enumerate(tickers):
    if i == 0:
        data = np.log(pd.DataFrame(yf.download(ticker,start_date,end_date))['Close'].rename(f'{ticker}')) - np.log(pd.DataFrame(yf.download(ticker))['Close'].rename(f'{ticker}').shift(1))
    else:
        iteration = np.log(pd.DataFrame(yf.download(ticker,start_date,end_date))['Close'].rename(f'{ticker}')) - np.log(pd.DataFrame(yf.download(ticker))['Close'].rename(f'{ticker}').shift(1))
        data = pd.merge(data,iteration,how = 'left',left_index= True,right_index= True)
        
data = data.dropna()
data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,PETR4.SA,VALE3.SA,ITUB4.SA,EGIE3.SA,^BVSP
Date,,,,,
2021-01-04,0.019913,0.044835,-0.023350,-0.008227,-0.006289
2021-01-05,0.038342,0.016698,-0.006493,-0.008989,0.005593
2021-01-06,0.001995,0.032269,0.028894,-0.008137,0.005254
2021-01-07,0.029462,0.063236,0.038186,-0.008439,0.017411
2021-01-08,0.003864,-0.003132,-0.000305,0.041048,0.025269
...,...,...,...,...,...
2021-12-23,0.006019,-0.009681,0.005168,-0.001812,-0.003360
2021-12-27,0.014716,-0.002530,0.010256,0.001295,0.006083
2021-12-28,0.001043,-0.024360,-0.003717,0.003873,-0.006340


In [43]:
indexes = data.index
omega = [0.25,0.25,0.25,0.25]
data["portfolio"] = np.zeros(len(data))

for index in indexes:
    for j,ticker in enumerate(tickers[:-1]):
        data.loc[index,["portfolio"]] +=  float(omega[j]) * np.exp(float(data.loc[index,[ticker]]))
    data.loc[index,["portfolio"]] = np.log(data.loc[index,["portfolio"]])

data = data[tickers[:-1]+["portfolio"]+tickers[-1:]]
data

,PETR4.SA,VALE3.SA,ITUB4.SA,EGIE3.SA,portfolio,^BVSP
Date,,,,,,
2021-01-04,0.019913,0.044835,-0.023350,-0.008227,0.008636,-0.006289
2021-01-05,0.038342,0.016698,-0.006493,-0.008989,0.010075,0.005593
2021-01-06,0.001995,0.032269,0.028894,-0.008137,0.013904,0.005254
2021-01-07,0.029462,0.063236,0.038186,-0.008439,0.030941,0.017411
2021-01-08,0.003864,-0.003132,-0.000305,0.041048,0.010530,0.025269
...,...,...,...,...,...,...
2021-12-23,0.006019,-0.009681,0.005168,-0.001812,-0.000057,-0.003360
2021-12-27,0.014716,-0.002530,0.010256,0.001295,0.005958,0.006083
2021-12-28,0.001043,-0.024360,-0.003717,0.003873,-0.005729,-0.006340


In [44]:
sigma = covariances_matrix_function(data)
tickers = ["PETR4","VALE3","ITUB4","EGIE3","portfolio"]
betas_log = dict.fromkeys(tickers,0)

for j,ticker in enumerate(tickers):
    betas_log[ticker] = float(sigma[j,len(sigma)-1]) / sigma[len(sigma)-1,len(sigma)-1]

In [45]:
sum_betas = 0
for j,ticker in enumerate(tickers[:-1]):
    sum_betas += omega[j] * betas_log[ticker]

In [46]:
print("Proportional sum of betas (log-returns): %f \n"%(sum_betas))
print("Portfolio's beta (log-returns): %f"%(betas_extra["portfolio"]))

Proportional sum of betas (log-returns): 0.966492 

Portfolio's beta (log-returns): 0.960687


In that case, we do not have linearity, by the very property of exponential and logarithmic functions. For small returns, the logarithmic and the linear one are quite close , but different enough, such that for $\;\; \epsilon = 0.001 \;,\; |\;\hat\beta - \sum\limits_{j=1}^{n}\omega_j \hat\beta_j| > \epsilon\;$.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=562772c3-9540-4081-b489-dda8173633f1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>